In [10]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from cesium import featurize
from cesium.time_series import TimeSeries

In [4]:
dataset_folder = "../all"
!ls {dataset_folder}

data_note.pdf	       test_set_metadata.csv  training_set_metadata.csv
sample_submission.csv  test_set_sample.csv
test_set.csv	       training_set.csv


In [5]:
metadata = pd.read_csv(f"{dataset_folder}/training_set_metadata.csv")
metadata.head(5)

,object_id,ra,decl,gal_l,gal_b,ddf,hostgal_specz,hostgal_photoz,hostgal_photoz_err,distmod,mwebv,target
0,615,349.046051,-61.943836,320.796530,-51.753706,1,0.0000,0.0000,0.0000,NaN,0.017,92
1,713,53.085938,-27.784405,223.525509,-54.460748,1,1.8181,1.6267,0.2552,45.4063,0.007,88
2,730,33.574219,-6.579593,170.455585,-61.548219,1,0.2320,0.2262,0.0157,40.2561,0.021,42
3,745,0.189873,-45.586655,328.254458,-68.969298,1,0.3037,0.2813,1.1523,40.7951,0.007,90
4,1124,352.711273,-63.823658,316.922299,-51.059403,1,0.1934,0.2415,0.0176,40.4166,0.024,90


In [6]:
train = pd.read_csv(f"{dataset_folder}/training_set.csv")
train.head(5)

,object_id,mjd,passband,flux,flux_err,detected
0,615,59750.4229,2,-544.810303,3.622952,1
1,615,59750.4306,1,-816.434326,5.553370,1
2,615,59750.4383,3,-471.385529,3.801213,1
3,615,59750.4450,4,-388.984985,11.395031,1
4,615,59752.4070,2,-681.858887,4.041204,1


In [7]:
features_to_use = ["amplitude",
                   "percent_beyond_1_std",
                   "maximum",
                   "max_slope",
                   "median",
                   "median_absolute_deviation",
                   "percent_close_to_median",
                   "minimum",
                   "skew",
                   "std",
                   "weighted_average"]

In [25]:
groups = train.groupby(['object_id', 'passband'])

In [30]:
times = groups.apply(lambda block: block['mjd'].values).reset_index().rename(columns={0: 'seq'})
times.head()

,object_id,passband,seq
0,615,0,"[59819.1532, 59820.1047, 59821.1026, 59822.110..."
1,615,1,"[59750.4306, 59752.4147, 59767.3045, 59770.225..."
2,615,2,"[59750.4229, 59752.407, 59767.2968, 59770.2179..."
3,615,3,"[59750.4383, 59752.4224, 59767.3122, 59770.233..."
4,615,4,"[59750.445, 59752.4334, 59767.3233, 59770.2445..."


In [31]:
flux = groups.apply(lambda block: block['flux'].values).reset_index().rename(columns={0: 'seq'})
flux.head()

,object_id,passband,seq
0,615,0,"[6.878784, 39.364853000000004, -10.422381, -65..."
1,615,1,"[-816.4343259999999, -1061.4570310000001, -815..."
2,615,2,"[-544.810303, -681.858887, -548.01355, -554.90..."
3,615,3,"[-471.385529, -524.95459, -475.51605199999995,..."
4,615,4,"[-388.984985, -393.480225, -405.663818, -400.2..."


In [32]:
err = groups.apply(lambda block: block['flux_err'].values).reset_index().rename(columns={0: 'seq'})
err.head()

,object_id,passband,seq
0,615,0,"[3.6331519999999995, 3.7756190000000003, 4.172..."
1,615,1,"[5.55337, 6.472994, 5.293019, 5.875329, 6.3068..."
2,615,2,"[3.6229519999999997, 4.0412040000000005, 3.462..."
3,615,3,"[3.8012129999999997, 3.552751, 3.340643, 3.736..."
4,615,4,"[11.395031, 3.5993459999999997, 3.496113, 3.83..."


In [64]:
fsets = {}
for object_id in metadata.object_id.unique():
    fset = featurize.featurize_time_series(times=times[times.object_id==object_id].seq.values,
                                           values=flux[flux.object_id==object_id].seq.values, 
                                           errors=err[err.object_id==object_id].seq.values,
                                           features_to_use=features_to_use)
    fsets[object_id] = fset

/home/frankl1/.env/plasticc/lib/python3.7/site-packages/cesium/features/amplitude.py:48: RuntimeWarning: overflow encountered in power
  linear_scale_data = base ** (exponent * x)
/home/frankl1/.env/plasticc/lib/python3.7/site-packages/cesium/features/amplitude.py:51: RuntimeWarning: invalid value encountered in double_scalars
  return (y_high - y_low) / (y_95 - y_5)
/home/frankl1/.env/plasticc/lib/python3.7/site-packages/cesium/features/amplitude.py:18: RuntimeWarning: overflow encountered in power
  linear_scale_data = base ** (exponent * x)
/home/frankl1/.env/plasticc/lib/python3.7/site-packages/cesium/features/cadence_features.py:12: RuntimeWarning: divide by zero encountered in true_divide
  return (cads[2:] + cads[:-2]) / (cads[1:-1] - cads[:-2])
/home/frankl1/.env/plasticc/lib/python3.7/site-packages/numpy/core/_methods.py:117: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
/home/frankl1/.env/plasticc/lib/python3.7/site-packages/cesium/feat

/home/frankl1/.env/plasticc/lib/python3.7/site-packages/cesium/features/cadence_features.py:12: RuntimeWarning: divide by zero encountered in true_divide
  return (cads[2:] + cads[:-2]) / (cads[1:-1] - cads[:-2])
/home/frankl1/.env/plasticc/lib/python3.7/site-packages/numpy/core/_methods.py:117: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
/home/frankl1/.env/plasticc/lib/python3.7/site-packages/cesium/features/cadence_features.py:12: RuntimeWarning: divide by zero encountered in true_divide
  return (cads[2:] + cads[:-2]) / (cads[1:-1] - cads[:-2])
/home/frankl1/.env/plasticc/lib/python3.7/site-packages/numpy/core/_methods.py:117: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
/home/frankl1/.env/plasticc/lib/python3.7/site-packages/cesium/features/cadence_features.py:12: RuntimeWarning: divide by zero encountered in true_divide
  return (cads[2:] + cads[:-2]) / (cads[1:-1] - cads[:-2])
/home/frankl1/.env/pl

/home/frankl1/.env/plasticc/lib/python3.7/site-packages/cesium/features/amplitude.py:48: RuntimeWarning: overflow encountered in power
  linear_scale_data = base ** (exponent * x)
/home/frankl1/.env/plasticc/lib/python3.7/site-packages/numpy/lib/function_base.py:3942: RuntimeWarning: invalid value encountered in multiply
  x2 = take(ap, indices_above, axis=axis) * weights_above
/home/frankl1/.env/plasticc/lib/python3.7/site-packages/cesium/features/amplitude.py:51: RuntimeWarning: invalid value encountered in double_scalars
  return (y_high - y_low) / (y_95 - y_5)
/home/frankl1/.env/plasticc/lib/python3.7/site-packages/cesium/features/amplitude.py:18: RuntimeWarning: overflow encountered in power
  linear_scale_data = base ** (exponent * x)
/home/frankl1/.env/plasticc/lib/python3.7/site-packages/cesium/features/amplitude.py:22: RuntimeWarning: invalid value encountered in double_scalars
  return max(abs((y_max - y_med) / y_med), abs((y_med - y_min) / y_med))
/home/frankl1/.env/plasticc

/home/frankl1/.env/plasticc/lib/python3.7/site-packages/cesium/features/cadence_features.py:12: RuntimeWarning: divide by zero encountered in true_divide
  return (cads[2:] + cads[:-2]) / (cads[1:-1] - cads[:-2])
/home/frankl1/.env/plasticc/lib/python3.7/site-packages/numpy/core/_methods.py:117: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
/home/frankl1/.env/plasticc/lib/python3.7/site-packages/cesium/features/cadence_features.py:12: RuntimeWarning: divide by zero encountered in true_divide
  return (cads[2:] + cads[:-2]) / (cads[1:-1] - cads[:-2])
/home/frankl1/.env/plasticc/lib/python3.7/site-packages/numpy/core/_methods.py:117: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
/home/frankl1/.env/plasticc/lib/python3.7/site-packages/cesium/features/cadence_features.py:12: RuntimeWarning: divide by zero encountered in true_divide
  return (cads[2:] + cads[:-2]) / (cads[1:-1] - cads[:-2])
/home/frankl1/.env/pl

/home/frankl1/.env/plasticc/lib/python3.7/site-packages/cesium/features/cadence_features.py:12: RuntimeWarning: divide by zero encountered in true_divide
  return (cads[2:] + cads[:-2]) / (cads[1:-1] - cads[:-2])
/home/frankl1/.env/plasticc/lib/python3.7/site-packages/numpy/core/_methods.py:117: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
/home/frankl1/.env/plasticc/lib/python3.7/site-packages/cesium/features/cadence_features.py:12: RuntimeWarning: divide by zero encountered in true_divide
  return (cads[2:] + cads[:-2]) / (cads[1:-1] - cads[:-2])
/home/frankl1/.env/plasticc/lib/python3.7/site-packages/numpy/core/_methods.py:117: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
/home/frankl1/.env/plasticc/lib/python3.7/site-packages/cesium/features/cadence_features.py:12: RuntimeWarning: divide by zero encountered in true_divide
  return (cads[2:] + cads[:-2]) / (cads[1:-1] - cads[:-2])
/home/frankl1/.env/pl

...

/home/frankl1/.env/plasticc/lib/python3.7/site-packages/numpy/lib/function_base.py:3826: RuntimeWarning: Invalid value encountered in percentile
  interpolation=interpolation)
/home/frankl1/.env/plasticc/lib/python3.7/site-packages/cesium/features/amplitude.py:48: RuntimeWarning: overflow encountered in power
  linear_scale_data = base ** (exponent * x)
/home/frankl1/.env/plasticc/lib/python3.7/site-packages/cesium/features/amplitude.py:51: RuntimeWarning: invalid value encountered in double_scalars
  return (y_high - y_low) / (y_95 - y_5)
/home/frankl1/.env/plasticc/lib/python3.7/site-packages/cesium/features/amplitude.py:18: RuntimeWarning: overflow encountered in power
  linear_scale_data = base ** (exponent * x)
/home/frankl1/.env/plasticc/lib/python3.7/site-packages/cesium/features/amplitude.py:22: RuntimeWarning: invalid value encountered in double_scalars
  return max(abs((y_max - y_med) / y_med), abs((y_med - y_min) / y_med))
/home/frankl1/.env/plasticc/lib/python3.7/site-packa

/home/frankl1/.env/plasticc/lib/python3.7/site-packages/cesium/features/amplitude.py:48: RuntimeWarning: overflow encountered in power
  linear_scale_data = base ** (exponent * x)
/home/frankl1/.env/plasticc/lib/python3.7/site-packages/cesium/features/amplitude.py:51: RuntimeWarning: invalid value encountered in double_scalars
  return (y_high - y_low) / (y_95 - y_5)
/home/frankl1/.env/plasticc/lib/python3.7/site-packages/cesium/features/amplitude.py:18: RuntimeWarning: overflow encountered in power
  linear_scale_data = base ** (exponent * x)
/home/frankl1/.env/plasticc/lib/python3.7/site-packages/cesium/features/amplitude.py:35: RuntimeWarning: overflow encountered in power
  linear_scale_data = base ** (exponent * x)
/home/frankl1/.env/plasticc/lib/python3.7/site-packages/numpy/lib/function_base.py:3826: RuntimeWarning: Invalid value encountered in percentile
  interpolation=interpolation)
/home/frankl1/.env/plasticc/lib/python3.7/site-packages/cesium/features/amplitude.py:48: Runti

/home/frankl1/.env/plasticc/lib/python3.7/site-packages/cesium/features/lomb_scargle.py:253: RuntimeWarning: invalid value encountered in sqrt
  out_dict['trend_error'] = np.sqrt(np.diag(vmodl0))
/home/frankl1/.env/plasticc/lib/python3.7/site-packages/cesium/features/lomb_scargle.py:256: RuntimeWarning: invalid value encountered in sqrt
  damp = np.sqrt(A0**2 * vA0 + B0**2 * vB0 + 2. * A0 * B0 * covA0B0) / amp
/home/frankl1/.env/plasticc/lib/python3.7/site-packages/cesium/features/lomb_scargle.py:266: RuntimeWarning: invalid value encountered in sqrt
  dphase[j] = np.sqrt(np.dot(np.dot(v, m), v))
/home/frankl1/.env/plasticc/lib/python3.7/site-packages/cesium/features/lomb_scargle.py:256: RuntimeWarning: invalid value encountered in sqrt
  damp = np.sqrt(A0**2 * vA0 + B0**2 * vB0 + 2. * A0 * B0 * covA0B0) / amp
/home/frankl1/.env/plasticc/lib/python3.7/site-packages/cesium/features/lomb_scargle.py:266: RuntimeWarning: invalid value encountered in sqrt
  dphase[j] = np.sqrt(np.dot(np.dot

/home/frankl1/.env/plasticc/lib/python3.7/site-packages/cesium/features/lomb_scargle.py:256: RuntimeWarning: invalid value encountered in sqrt
  damp = np.sqrt(A0**2 * vA0 + B0**2 * vB0 + 2. * A0 * B0 * covA0B0) / amp
/home/frankl1/.env/plasticc/lib/python3.7/site-packages/cesium/features/lomb_scargle.py:266: RuntimeWarning: invalid value encountered in sqrt
  dphase[j] = np.sqrt(np.dot(np.dot(v, m), v))
/home/frankl1/.env/plasticc/lib/python3.7/site-packages/cesium/features/amplitude.py:48: RuntimeWarning: overflow encountered in power
  linear_scale_data = base ** (exponent * x)
/home/frankl1/.env/plasticc/lib/python3.7/site-packages/numpy/lib/function_base.py:3942: RuntimeWarning: invalid value encountered in multiply
  x2 = take(ap, indices_above, axis=axis) * weights_above
/home/frankl1/.env/plasticc/lib/python3.7/site-packages/cesium/features/amplitude.py:18: RuntimeWarning: overflow encountered in power
  linear_scale_data = base ** (exponent * x)
/home/frankl1/.env/plasticc/lib

/home/frankl1/.env/plasticc/lib/python3.7/site-packages/cesium/features/amplitude.py:48: RuntimeWarning: overflow encountered in power
  linear_scale_data = base ** (exponent * x)
/home/frankl1/.env/plasticc/lib/python3.7/site-packages/cesium/features/amplitude.py:18: RuntimeWarning: overflow encountered in power
  linear_scale_data = base ** (exponent * x)
/home/frankl1/.env/plasticc/lib/python3.7/site-packages/cesium/features/amplitude.py:35: RuntimeWarning: overflow encountered in power
  linear_scale_data = base ** (exponent * x)
/home/frankl1/.env/plasticc/lib/python3.7/site-packages/cesium/features/lomb_scargle.py:253: RuntimeWarning: invalid value encountered in sqrt
  out_dict['trend_error'] = np.sqrt(np.diag(vmodl0))
/home/frankl1/.env/plasticc/lib/python3.7/site-packages/cesium/features/lomb_scargle.py:256: RuntimeWarning: invalid value encountered in sqrt
  damp = np.sqrt(A0**2 * vA0 + B0**2 * vB0 + 2. * A0 * B0 * covA0B0) / amp
/home/frankl1/.env/plasticc/lib/python3.7/site

In [65]:
df_fset = pd.concat(fsets)
df_fset

feature         amplitude percent_beyond_1_std       maximum    max_slope  \
channel                 0                    0             0            0   
615       0    121.048016             0.476190    125.182808   184.863919   
          1    880.533203             0.586207    660.626343   594.695677   
          2    646.921722             0.568966    611.984558   423.361104   
          3    488.190827             0.551724    445.737061   280.055192   
          4    402.069122             0.517241    381.953735   205.475911   
          5    400.501618             0.596491    378.188141   226.111645   
713       0     14.622504             0.385714     14.509829    11.418733   
          1     10.422385             0.392857      9.129021     1.738723   
          2     10.298480             0.410714     10.529041     1.798635   
          3     11.862454             0.410714     11.330316     2.730983   
          4     11.057367             0.375000      9.827934     3.607656   
          5     14.491025             0.392857     14.770886     6.128550   
730       0      4.701063             0.333333      5.942166     5.987811   
          1      4.543095             0.442308      5.693109     2.436376   
          2     11.921774             0.134615     20.994711     2.128031   
          3     19.503951             0.115385     33.572102     3.727682   
          4     23.498145             0.098039     41.159981     3.045581   
          5     33.234935             0.215686     47.310059     5.935501   
745       0     10.944189             0.180556     18.014029    13.567209   
          1     97.931352             0.035714    192.244293     5.682263   
          2    111.477483             0.071429    220.795212     4.867642   
          3    104.097369             0.089286    203.250702     3.393333   
          4     99.563790             0.107143    183.633118     3.227811   
          5     75.881338             0.090909    141.513290     5.749368   
1124      0      6.067815             0.333333      5.330927     6.992729   
          1     19.896143             0.172414     37.170177     2.127987   
          2     54.378113             0.137931    106.671692     4.647070   
          3     71.309338             0.103448    139.818405     7.223411   
          4     80.071971             0.086207    143.600189     8.318557   
          5     60.009063             0.052632    109.157585     6.676510   
...                   ...                  ...           ...          ...   
130739978 0     37.357846             0.117647     51.247578    72.694776   
          1     80.459251             0.071429    157.191650     5.765441   
          2     22.959684             0.083333     38.534805     4.130648   
          3     17.285638             0.360000     23.924580    16.405137   
          4     40.893879             0.375000     46.577839    24.384630   
          5    311.488880             0.142857    517.602478   131.124321   
130755807 0    147.049413             0.083333    271.930481    11.243460   
          1     28.318704             0.090909     54.464066     0.690205   
          2      8.846144             0.500000      8.301914     3.567556   
          3    180.521552             0.071429    329.695709    15.345343   
          4    203.362503             0.120000    363.402466    16.204538   
          5    102.181900             0.382353    135.327408    63.702486   
130762946 0     93.877626             0.230769     52.152622    83.552077   
          1     49.707789             0.352941      9.115225    20.998419   
          2     54.212563             0.416667     27.459307    16.085210   
          3     65.235669             0.352941     60.889641    54.995158   
          4     43.384130             0.375000     37.266300    52.691937   
          5    128.713696             0.290323    169.916672    72.293669   
130772921 0     24.886840             0.214286     37.959984    18.941907

In [93]:
df_fset.columns = df_fset.columns.droplevel(1)
df_fset

feature         amplitude  percent_beyond_1_std       maximum    max_slope  \
615       0    121.048016              0.476190    125.182808   184.863919   
          1    880.533203              0.586207    660.626343   594.695677   
          2    646.921722              0.568966    611.984558   423.361104   
          3    488.190827              0.551724    445.737061   280.055192   
          4    402.069122              0.517241    381.953735   205.475911   
          5    400.501618              0.596491    378.188141   226.111645   
713       0     14.622504              0.385714     14.509829    11.418733   
          1     10.422385              0.392857      9.129021     1.738723   
          2     10.298480              0.410714     10.529041     1.798635   
          3     11.862454              0.410714     11.330316     2.730983   
          4     11.057367              0.375000      9.827934     3.607656   
          5     14.491025              0.392857     14.770886     6.128550   
730       0      4.701063              0.333333      5.942166     5.987811   
          1      4.543095              0.442308      5.693109     2.436376   
          2     11.921774              0.134615     20.994711     2.128031   
          3     19.503951              0.115385     33.572102     3.727682   
          4     23.498145              0.098039     41.159981     3.045581   
          5     33.234935              0.215686     47.310059     5.935501   
745       0     10.944189              0.180556     18.014029    13.567209   
          1     97.931352              0.035714    192.244293     5.682263   
          2    111.477483              0.071429    220.795212     4.867642   
          3    104.097369              0.089286    203.250702     3.393333   
          4     99.563790              0.107143    183.633118     3.227811   
          5     75.881338              0.090909    141.513290     5.749368   
1124      0      6.067815              0.333333      5.330927     6.992729   
          1     19.896143              0.172414     37.170177     2.127987   
          2     54.378113              0.137931    106.671692     4.647070   
          3     71.309338              0.103448    139.818405     7.223411   
          4     80.071971              0.086207    143.600189     8.318557   
          5     60.009063              0.052632    109.157585     6.676510   
...                   ...                   ...           ...          ...   
130739978 0     37.357846              0.117647     51.247578    72.694776   
          1     80.459251              0.071429    157.191650     5.765441   
          2     22.959684              0.083333     38.534805     4.130648   
          3     17.285638              0.360000     23.924580    16.405137   
          4     40.893879              0.375000     46.577839    24.384630   
          5    311.488880              0.142857    517.602478   131.124321   
130755807 0    147.049413              0.083333    271.930481    11.243460   
          1     28.318704              0.090909     54.464066     0.690205   
          2      8.846144              0.500000      8.301914     3.567556   
          3    180.521552              0.071429    329.695709    15.345343   
          4    203.362503              0.120000    363.402466    16.204538   
          5    102.181900              0.382353    135.327408    63.702486   
130762946 0     93.877626              0.230769     52.152622    83.552077   
          1     49.707789              0.352941      9.115225    20.998419   
          2     54.212563              0.416667     27.459307    16.085210   
          3     65.235669              0.352941     60.889641    54.995158   
          4     43.384130              0.375000     37.266300    52.691937   
          5    128.713696              0.290323    169.916672    72.293669   
130772921 0     24.886840              0.214286     37.959984    18.941907   
          1    163.773

In [94]:
df_fset.to_csv("feature_set.csv")

In [95]:
pd.read_csv("feature_set.csv", index_col=[0, 1])

amplitude  percent_beyond_1_std       maximum    max_slope  \
615       0    121.048016              0.476190    125.182808   184.863919   
          1    880.533203              0.586207    660.626343   594.695677   
          2    646.921722              0.568966    611.984558   423.361104   
          3    488.190826              0.551724    445.737061   280.055192   
          4    402.069122              0.517241    381.953735   205.475911   
          5    400.501618              0.596491    378.188141   226.111645   
713       0     14.622504              0.385714     14.509829    11.418733   
          1     10.422385              0.392857      9.129021     1.738723   
          2     10.298480              0.410714     10.529041     1.798635   
          3     11.862454              0.410714     11.330316     2.730983   
          4     11.057367              0.375000      9.827934     3.607656   
          5     14.491025              0.392857     14.770886     6.128550   
730       0      4.701063              0.333333      5.942166     5.987811   
          1      4.543095              0.442308      5.693109     2.436376   
          2     11.921774              0.134615     20.994711     2.128031   
          3     19.503950              0.115385     33.572102     3.727682   
          4     23.498145              0.098039     41.159981     3.045581   
          5     33.234935              0.215686     47.310059     5.935501   
745       0     10.944189              0.180556     18.014029    13.567209   
          1     97.931352              0.035714    192.244293     5.682263   
          2    111.477482              0.071429    220.795212     4.867642   
          3    104.097369              0.089286    203.250702     3.393333   
          4     99.563790              0.107143    183.633118     3.227811   
          5     75.881338              0.090909    141.513290     5.749368   
1124      0      6.067815              0.333333      5.330927     6.992729   
          1     19.896143              0.172414     37.170177     2.127987   
          2     54.378113              0.137931    106.671692     4.647070   
          3     71.309338              0.103448    139.818405     7.223411   
          4     80.071971              0.086207    143.600189     8.318557   
          5     60.009063              0.052632    109.157585     6.676510   
...                   ...                   ...           ...          ...   
130739978 0     37.357846              0.117647     51.247578    72.694776   
          1     80.459251              0.071429    157.191650     5.765441   
          2     22.959684              0.083333     38.534805     4.130648   
          3     17.285638              0.360000     23.924580    16.405137   
          4     40.893879              0.375000     46.577839    24.384630   
          5    311.488880              0.142857    517.602478   131.124321   
130755807 0    147.049414              0.083333    271.930481    11.243460   
          1     28.318704              0.090909     54.464066     0.690205   
          2      8.846144              0.500000      8.301914     3.567556   
          3    180.521552              0.071429    329.695709    15.345343   
          4    203.362503              0.120000    363.402466    16.204538   
          5    102.181900              0.382353    135.327408    63.702486   
130762946 0     93.877627              0.230769     52.152622    83.552077   
          1     49.707790              0.352941      9.115225    20.998419   
          2     54.212564              0.416667     27.459307    16.085210   
          3     65.235669              0.352941     60.889641    54.995158   
          4     43.384130              0.375000     37.266300    52.691937   
          5    128.713696              0.290323    169.916672    72.293669   
130772921 0     24.886840              0.214286     37.959984    18.941907   
          1    163.773464             